In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
# library(gt)
library(car)
install.packages("stargazer")
library(stargazer)
# First install and load lmerTest
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpsgwYGP/downloaded_packages

The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpsgwYGP/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpsgwYGP/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpsgwYGP/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [2]:

file_path_total <- 'pilot_total_stats/distractor_wm_ltm.csv'
file_path_5 <- 'pilot5_cubic_n_split_stats/distractor_wm_ltm.csv'
file_path_6 <- 'pilot6_cubic_n_split_stats/distractor_wm_ltm.csv'

df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 


In [3]:
control <- lmerControl(optimizer = "bobyqa")#, optCtrl = list(maxfun = 10000))

In [16]:
run_ltm_models <- function(df) {
    
    it_third_test <- poly(df$it_sim_dis_diff_test, 3, raw = FALSE)
    v2_third_test <- poly(df$v2_sim_dis_diff_test, 3, raw = FALSE)

    
    it_third_test_sign <- poly(df$it_sim_dis_diff_test_sign, 3, raw = FALSE)
    v2_third_test_sign <- poly(df$v2_sim_dis_diff_test_sign, 3, raw = FALSE)

    df$it_sim_dis_diff_test_z <- it_third_test_sign[, 1]
    df$it_sim_dis_diff_test_sq_z   <- it_third_test_sign[, 2]
    df$it_sim_dis_diff_test_cube_z <- it_third_test_sign[, 3]

    df$v2_sim_dis_diff_test_z  <- v2_third_test_sign[, 1]
    df$v2_sim_dis_diff_test_sq_z   <- v2_third_test_sign[, 2]
    df$v2_sim_dis_diff_test_cube_z <- v2_third_test_sign[, 3]

    df$it_sim_dis_diff_test_sign_z <- it_third_test_sign[, 1]
    df$it_sim_dis_diff_test_sign_sq_z   <- it_third_test_sign[, 2]
    df$it_sim_dis_diff_test_sign_cube_z <- it_third_test_sign[, 3]

    df$v2_sim_dis_diff_test_sign_z  <- v2_third_test_sign[, 1]
    df$v2_sim_dis_diff_test_sign_sq_z   <- v2_third_test_sign[, 2]
    df$v2_sim_dis_diff_test_sign_cube_z <- v2_third_test_sign[, 3]
    
    model_abs <- glmer(recall_answ ~ (it_sim_dis_test_z + it_sim_dis_test_sq_z + it_sim_dis_test_cube_z)  
                      + (v2_sim_dis_test_z + v2_sim_dis_test_sq_z + v2_sim_dis_test_cube_z)  
                        + reliability_binary_z *  validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
    
    
    model_rel <- glmer(recall_answ ~ (it_sim_dis_diff_test_z + it_sim_dis_diff_test_sq_z + it_sim_dis_diff_test_cube_z) 
                       + (v2_sim_dis_diff_test_z + v2_sim_dis_diff_test_sq_z + v2_sim_dis_diff_test_cube_z) 
                       + reliability_binary_z * validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))

    model_rel_split <- glmer(recall_answ ~ (it_sim_dis_diff_test_sign_z + it_sim_dis_diff_test_sign_sq_z + it_sim_dis_diff_test_sign_cube_z) * it_pos_neg 
                             + (v2_sim_dis_diff_test_sign_z + v2_sim_dis_diff_test_sign_sq_z + v2_sim_dis_diff_test_sign_cube_z) * v2_pos_neg 
                             + reliability_binary_z * validity_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
return(list( 
    model_abs       = model_abs,
    model_rel       = model_rel,
    model_rel_split = model_rel_split
  ))
}



In [17]:
results_old_stims <- run_ltm_models(df6)
results_new_stims <- run_ltm_models(df5)
results_total <- run_ltm_models(df_total)





In [18]:
extract_significance <- function(model_summary) {
  # For glmer models
  if ("coefficients" %in% names(model_summary)) {
    # Check more carefully for z value vs t value columns
    coef_table <- model_summary$coefficients
    var_names <- rownames(coef_table)
    
    # Check which p-value column exists
    if ("Pr(>|z|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|z|)"]
    } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|t|)"]
    } else {
      # If there's a z or t value, calculate p-values manually
      if ("z value" %in% colnames(coef_table)) {
        z_values <- coef_table[, "z value"]
        p_values <- 2 * pnorm(abs(z_values), lower.tail = FALSE)
      } else if ("t value" %in% colnames(coef_table)) {
        t_values <- coef_table[, "t value"]
        # Using a large df for approximation
        p_values <- 2 * pt(abs(t_values), df = 100, lower.tail = FALSE)
      } else {
        stop("Cannot find p-values or z/t values in coefficients table")
      }
    }
  } else {
    # More detailed debug info
    cat("Available names in model summary:\n")
    print(names(model_summary))
    
    # Try to access fixed effects via other means
    if ("fixed.effects" %in% names(model_summary)) {
      var_names <- names(model_summary$fixed.effects)
      # Manually calculate p-values using fixed effects and std errors
      # This is just a placeholder - would need to be adapted to actual structure
      stop("Found fixed.effects but need appropriate calculation of p-values")
    } else {
      stop("Could not extract coefficients from model summary")
    }
  }
  
  # Calculate significance stars
  significance <- sapply(p_values, function(p) {
    if (is.na(p)) return("")
    if (p < 0.001) return("***")
    if (p < 0.01) return("**")
    if (p < 0.05) return("*")
    if (p < 0.1) return(".")
    return("")
  })
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}


In [19]:
compare_models <- function(model_list, model_names, 
                          include_p_values = FALSE,
                          only_significant = TRUE,
                          significance_level = 0.1) {
  
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # First, determine which rows have significance in any model
  has_significance_by_var <- sapply(all_vars, function(var) {
    any_significant <- FALSE
    for(model_name in model_names) {
      model_sig <- sig_list[[model_name]]
      idx <- which(model_sig$variable == var)
      if(length(idx) > 0) {
        stars <- model_sig$significance[idx]
        if(stars != "") {
          any_significant <- TRUE
          break
        }
      }
    }
    return(any_significant)
  })
  
  # Now populate the data frame with all p-values for rows with any significance
  for(model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if(length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      # Always show p-value if this variable has significance in any model
      if(has_significance_by_var[var] || !only_significant) {
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else if(stars != "") {
        # Show significance stars with p-value for significant results
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else {
        # For non-significant results in rows without any significance
        return("")
      }
    })
  }
  
  # Filter rows if only_significant is TRUE
  if(only_significant) {
    result_df <- result_df[has_significance_by_var, ]
  }
  
  # Adjust HTML table styling to make first column wider
  html_table <- htmlTable(
    result_df,
    caption = "Significance Comparison Across Models",
    rnames = FALSE,
    css.cell = "padding: 8px;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold; border: 1px solid #dddddd;",
    css.rgroup = "border-bottom: 1px solid #dddddd;",
    css.tspanner = "font-weight: bold; text-align: center; border-bottom: 2px solid #dddddd;",
    css.class = "table-hover"
  )
  
  # Add additional CSS to make the first column wider
  html_table <- gsub("<table", "<style>td:first-child, th:first-child { min-width: 300px; max-width: 500px; word-wrap: break-word; }</style><table", html_table)
  
  return(html_table)
}

In [20]:

model_abs_old <- summary(results_old_stims$model_abs)
model_abs_new <- summary(results_new_stims$model_abs)
model_abs_total <- summary(results_total$model_abs)

comparison_table_abs <- compare_models(
  model_list = list(model_abs_old, model_abs_new, model_abs_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_abs, "comparison_table_distractor_abs_cent.html")



model_rel_old <- summary(results_old_stims$model_rel)
model_rel_new <- summary(results_new_stims$model_rel)
model_rel_total <- summary(results_total$model_rel)

comparison_table_rel <- compare_models(
  model_list = list(model_rel_old, model_rel_new, model_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_rel, "comparison_table_distractor_rel_cent.html")



model_rel_split_old <- summary(results_old_stims$model_rel_split)
model_rel_split_new <- summary(results_new_stims$model_rel_split)
model_rel_split_total <- summary(results_total$model_rel_split)

comparison_table_rel_split <- compare_models(
  model_list = list(model_rel_split_old, model_rel_split_new, model_rel_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
# writeLines(comparison_table_rel_split, "comparison_table_distractor_rel_split_cent.html")
writeLines(comparison_table_abs, "comparison_table_distractor_abs_rt.html")
writeLines(comparison_table_rel, "comparison_table_distractor_rel_rt.html")
writeLines(comparison_table_rel_split, "comparison_table_distractor_rel_split_rt.html")


In [21]:
# IRdisplay::display_html(comparison_table_abs_split)
IRdisplay::display_html(comparison_table_abs)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_test_z 
 *** (p=0.0007) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_test_sq_z 
 * (p=0.0264) 
 ** (p=0.0019) 
 *** (p=0.0001) 
 
 
 it_sim_dis_test_cube_z 
 *** (p=0.0001) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 v2_sim_dis_test_sq_z 
 . (p=0.0689) 
 (p=0.7766) 
 (p=0.1894)

In [22]:
IRdisplay::display_html(comparison_table_rel)
IRdisplay::display_html(comparison_table_rel_split)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_z 
 * (p=0.0471) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sq_z 
 *** (p=0.0000) 
 . (p=0.0982) 
 ** (p=0.0012) 
 
 
 v2_sim_dis_diff_test_z 
 ** (p=0.0026) 
 (p=0.3716) 
 ** (p=0.0011) 
 
 
 v2_sim_dis_diff_test_sq_z 
 (p=0.4382) 
 . (p=0.0719) 
 (p=0.4903) 
 
 
 v2_sim_dis_diff_test_cube_z 
 (p=0.7783) 
 * (p=0.0220) 
 * (p=0.0425)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_z 
 . (p=0.0502) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_sq_z 
 *** (p=0.0000) 
 (p=0.1017) 
 ** (p=0.0013) 
 
 
 it_pos_neg 
 (p=0.7634) 
 . (p=0.0826) 
 (p=0.1363) 
 
 
 v2_sim_dis_diff_test_sign_z 
 ** (p=0.0015) 
 (p=0.3117) 
 *** (p=0.0004) 
 
 
 v2_sim_dis_diff_test_sign_sq_z 
 (p=0.4207) 
 . (p=0.0872) 
 (p=0.5052) 
 
 
 v2_sim_dis_diff_test_sign_cube_z 
 (p=0.7298) 
 * (p=0.0271) 
 . (p=0.0536) 
 
 
 it_sim_dis_diff_test_sign_z:it_pos_neg 
 (p=0.3130) 
 ** (p=0.0047) 
 ** (p=0.0051) 
 
 
 v2_sim_dis_diff_test_sign_cube_z:v2_pos_neg 
 (p=0.1385) 
 ** (p=0.0018) 
 *** (p=0.0009)

In [23]:
library(knitr)
library(kableExtra)
library(IRdisplay)

# Function to create combined results table with colors
create_combined_results_table <- function(model_old, model_new, model_total, 
                                          model_name = "Combined Model Results",
                                          save_file = NULL) {
  
  # Function to clean predictor names
  clean_names <- function(names) {
    names <- gsub("validity_binary_z", "validity", names)
    names <- gsub("reliability_binary_z", "reliability", names)
    # Order matters! Do cube and square before the base term
    names <- gsub("it_sim_dis_diff_test_cube_z", "IT_cubic", names)
    names <- gsub("it_sim_dis_diff_test_sq_z", "IT_quadratic", names)
    names <- gsub("it_sim_dis_diff_test_z", "IT_linear", names)
    names <- gsub("v2_sim_dis_diff_test_cube_z", "V2_cubic", names)
    names <- gsub("v2_sim_dis_diff_test_sq_z", "V2_quadratic", names)
    names <- gsub("v2_sim_dis_diff_test_z", "V2_linear", names)
    names <- gsub("it_pos_neg", "IT_sign", names)
    names <- gsub("v2_pos_neg", "V2_sign", names)
    return(names)
  }
  
  # Function to format p-value (vectorized, no leading zero)
  format_p <- function(p_vec) {
    sapply(p_vec, function(p) {
      if (p < 0.001) {
        return("&lt;.001")
      } else {
        formatted <- sprintf("%.3f", p)
        # Remove leading zero
        formatted <- sub("^0\\.", ".", formatted)
        return(formatted)
      }
    })
  }
  
  # Function to format to n significant figures
  format_sig_figs <- function(x, n = 2) {
    formatted <- signif(x, n)
    # Convert to character, preserving the significant figures
    as.character(formatted)
  }
  
  # Function to combine values with color formatting
  combine_colored <- function(old_val, new_val, total_val) {
    paste0(
      '<span style="color: purple;">', old_val, '</span>',
      ' / ',
      '<span style="color: blue;">', new_val, '</span>',
      ' / ',
      '<span style="color: black; font-weight: bold;">', total_val, '</span>'
    )
  }
  
  # Get coefficients for all three models
  coef_old <- coef(summary(model_old))
  coef_new <- coef(summary(model_new))
  coef_total <- coef(summary(model_total))
  
  # Get confidence intervals
  ci_old <- confint(model_old, method = "Wald", quiet = TRUE)
  ci_new <- confint(model_new, method = "Wald", quiet = TRUE)
  ci_total <- confint(model_total, method = "Wald", quiet = TRUE)
  
  # Get predictor names
  predictors <- clean_names(rownames(coef_total))
  
  # Check which predictors are significant in all three models (p < .05)
  sig_in_all <- (coef_old[, "Pr(>|z|)"] < 0.05) & 
                (coef_new[, "Pr(>|z|)"] < 0.05) & 
                (coef_total[, "Pr(>|z|)"] < 0.05)
  
  # Bold predictor names that are significant in all three
  predictors_formatted <- ifelse(sig_in_all, 
                                 paste0("<strong>", predictors, "</strong>"), 
                                 predictors)
  
  # Combine values with color formatting
  results <- data.frame(
    Predictor = predictors_formatted,
    Beta = combine_colored(
      format_sig_figs(coef_old[, "Estimate"], 2),
      format_sig_figs(coef_new[, "Estimate"], 2),
      format_sig_figs(coef_total[, "Estimate"], 2)
    ),
    SE = combine_colored(
      format_sig_figs(coef_old[, "Std. Error"], 2),
      format_sig_figs(coef_new[, "Std. Error"], 2),
      format_sig_figs(coef_total[, "Std. Error"], 2)
    ),
    z = combine_colored(
      format_sig_figs(coef_old[, "z value"], 2),
      format_sig_figs(coef_new[, "z value"], 2),
      format_sig_figs(coef_total[, "z value"], 2)
    ),
    p = combine_colored(
      format_p(coef_old[, "Pr(>|z|)"]),
      format_p(coef_new[, "Pr(>|z|)"]),
      format_p(coef_total[, "Pr(>|z|)"])
    ),
    CI_Lower = combine_colored(
      format_sig_figs(ci_old[rownames(coef_old), 1], 2),
      format_sig_figs(ci_new[rownames(coef_new), 1], 2),
      format_sig_figs(ci_total[rownames(coef_total), 1], 2)
    ),
    CI_Upper = combine_colored(
      format_sig_figs(ci_old[rownames(coef_old), 2], 2),
      format_sig_figs(ci_new[rownames(coef_new), 2], 2),
      format_sig_figs(ci_total[rownames(coef_total), 2], 2)
    ),
    stringsAsFactors = FALSE
  )
  
  # Create formatted table with borders
  table <- kable(results, row.names = FALSE, align = c("l", "r", "r", "r", "r", "r", "r"),
                 escape = FALSE, format = "html") %>%
    kable_styling(bootstrap_options = c("striped", "hover", "condensed", "bordered"),
                  full_width = FALSE) %>%
    add_header_above(c(" " = 1, 
                       "Study 1 / Study 2 / Combined" = 6))
  
  # Create header
  header_html <- paste0(
    '<h3>', model_name, '</h3>',
    '<p style="font-size: 12px;">',
    '<span style="color: purple;">■</span> Study 1 &nbsp;&nbsp; ',
    '<span style="color: blue;">■</span> Study 2 &nbsp;&nbsp; ',
    '<span style="color: black; font-weight: bold;">■</span> Combined',
    '<br><strong>Bold predictors</strong> are significant (p &lt; .05) in all three studies.',
    '</p>'
  )
  
  # Add custom CSS for clear borders
  css_style <- '
  <style>
    table {
      border-collapse: collapse !important;
      border: 2px solid #333 !important;
    }
    th, td {
      border: 1px solid #666 !important;
      padding: 8px !important;
    }
    th {
      background-color: #f0f0f0 !important;
      font-weight: bold !important;
    }
  </style>
  '
  
  # Combine HTML
  full_html <- paste0(css_style, header_html, as.character(table))
  
  # Display in notebook
  display_html(full_html)
  
  # Save to file if requested
  if (!is.null(save_file)) {
    complete_html <- paste0(
      '<!DOCTYPE html>
      <html>
      <head>
        <meta charset="utf-8">
        <title>', model_name, '</title>',
      css_style,
      '</head>
      <body>',
      full_html,
      '</body>
      </html>'
    )
    writeLines(complete_html, save_file)
    cat("\nTable saved to:", save_file, "\n")
  }
  
  return(results)
}


ltm_results_old <- run_ltm_models(df6)     
ltm_results_new <- run_ltm_models(df5)      
ltm_results_total <- run_ltm_models(df_total)  

ltm_rel_split_combined <- create_combined_results_table(
  ltm_results_old$model_rel,
  ltm_results_new$model_rel,
  ltm_results_total$model_rel,
  "Model Results: LTM Recall",
  save_file = "ltm_rel_results_distractor.html"
)

Warning message:
“'xfun::attr()' is deprecated.
Use 'xfun::attr2()' instead.
See help("Deprecated")”


Model Results: LTM Recall ■ Study 1    ■ Study 2    ■ Combined Bold predictors are significant (p < .05) in all three studies. 
 
 
 
 Study 1 / Study 2 / Combined 
 
 
 Predictor 
 Beta 
 SE 
 z 
 p 
 CI_Lower 
 CI_Upper 
 
 
 
 
 (Intercept) 
 -0.3 / -0.28 / -0.29 
 0.066 / 0.062 / 0.045 
 -4.6 / -4.5 / -6.5 
 <.001 / <.001 / <.001 
 -0.43 / -0.4 / -0.38 
 -0.18 / -0.16 / -0.2 
 
 
 IT_linear 
 4.2 / 14 / 13 
 2.1 / 2.1 / 2.1 
 2 / 6.8 / 6.3 
 .047 / <.001 / <.001 
 0.054 / 10 / 9.2 
 8.4 / 19 / 17 
 
 
 IT_quadratic 
 14 / -3.5 / 6.9 
 2.2 / 2.1 / 2.1 
 6.6 / -1.7 / 3.2 
 <.001 / .098 / .001 
 10 / -7.7 / 2.7 
 19 / 0.66 / 11 
 
 
 IT_cubic 
 1.2 / -1.6 / -0.017 
 2.1 / 2.1 / 2.1 
 0.58 / -0.74 / -0.0079 
 .561 / .460 / .994 
 -2.9 / -5.8 / -4.2 
 5.4 / 2.6 / 4.1 
 
 
 V2_linear 
 6.5 / 1.9 / 6.9 
 2.2 / 2.1 / 2.1 
 3 / 0.89 / 3.3 
 .003 / .372 / .001 
 2.3 / -2.3 / 2.8 
 11 / 6 / 11 
 
 
 V2_quadratic 
 -1.7 / 3.8 / 1.5 
 2.1 / 2.1 / 2.1 
 -0.78 / 1.8 / 0.69 
 .438 / .072 / .490 
 -5.9 / -0.34 / -2.7 
 2.5 / 8 / 5.6 
 
 
 V2_cubic 
 0.62 / -4.8 / -4.3 
 2.2 / 2.1 / 2.1 
 0.28 / -2.3 / -2 
 .778 / .022 / .042 
 -3.7 / -9 / -8.5 
 4.9 / -0.7 / -0.15 
 
 
 reliability 
 -0.0086 / -0.028 / -0.015 
 0.035 / 0.033 / 0.024 
 -0.24 / -0.84 / -0.64 
 .807 / .400 / .521 
 -0.078 / -0.092 / -0.062 
 0.061 / 0.037 / 0.032 
 
 
 validity 
 0.023 / 0.011 / 0.018 
 0.037 / 0.034 / 0.025 
 0.63 / 0.33 / 0.7 
 .526 / .745 / .483 
 -0.049 / -0.056 / -0.031 
 0.095 / 0.078 / 0.066 
 
 
 reliability:validity 
 7e-05 / -0.048 / -0.021 
 0.037 / 0.034 / 0.025 
 0.0019 / -1.4 / -0.86 
 .998 / .159 / .392 
 -0.072 / -0.12 / -0.07 
 0.072 / 0.019 / 0.028


Table saved to: ltm_rel_results_distractor.html 


In [28]:
# Change the modeling function from glmer (for binary data) to lmer (for continuous data)
run_rt_models <- function(df) {
    # First, let's rename the dataset column to make it clearer
    df$rt_ltm <- df$image_recall_response.rt
    df_correct <- subset(df, recall_answ == 1)
    df_correct$log_rt_ltm <- log(df_correct$rt_ltm)

    cols_to_scale <- c("reliability_binary_z", "validity_binary_z", 
                  "it_sim_dis_test_z", "it_sim_dis_test_sq_z","it_sim_dis_test_cube_z", "v2_sim_dis_test_z", "v2_sim_dis_test_sq_z","v2_sim_dis_test_cube_z",
                   "it_sim_dis_diff_test_z", "it_sim_dis_diff_test_sq_z","it_sim_dis_diff_test_cube_z", "v2_sim_dis_diff_test_z", "v2_sim_dis_diff_test_sq_z","v2_sim_dis_diff_test_cube_z", 
                      "it_sim_dis_diff_test_sign_z", "it_sim_dis_diff_test_sign_sq_z","it_sim_dis_diff_test_sign_cube_z", "v2_sim_dis_diff_test_sign_z", "v2_sim_dis_diff_test_sign_sq_z", "v2_sim_dis_diff_test_sign_cube_z")

    df_correct[paste0(cols_to_scale, "_new")] <- lapply(df_correct[cols_to_scale], scale)

    it_sim_dis_diff_test_z_rt <- poly(df_correct$it_sim_dis_diff_test_z, 3, raw = FALSE)
    v2_sim_dis_diff_test_z_rt <- poly(df_correct$v2_sim_dis_diff_test_z, 3, raw = FALSE)
    
    it_sim_dis_diff_test_sign_z_rt <- poly(df_correct$it_sim_dis_diff_test_sign_z, 3, raw = FALSE)
    v2_sim_dis_diff_test_sign_z_rt <- poly(df_correct$v2_sim_dis_diff_test_sign_z, 3, raw = FALSE)

    df_correct$it_sim_dis_diff_test_z_new  <- it_sim_dis_diff_test_z_rt[, 1]
    df_correct$it_sim_dis_diff_test_sq_z_new   <- it_sim_dis_diff_test_z_rt[, 2]
    df_correct$it_sim_dis_diff_test_cube_z_new <- it_sim_dis_diff_test_z_rt[, 3]


    df_correct$v2_sim_dis_diff_test_z_new  <- v2_sim_dis_diff_test_z_rt[, 1]
    df_correct$v2_sim_dis_diff_test_sq_z_new   <- v2_sim_dis_diff_test_z_rt[, 2]
    df_correct$v2_sim_dis_diff_test_cube_z_new <- v2_sim_dis_diff_test_z_rt[, 3]

    
    df_correct$it_sim_dis_diff_test_sign_z_new  <- it_sim_dis_diff_test_sign_z_rt[, 1]
    df_correct$it_sim_dis_diff_test_sign_sq_z_new   <- it_sim_dis_diff_test_sign_z_rt[, 2]
    df_correct$it_sim_dis_diff_test_sign_cube_z_new <- it_sim_dis_diff_test_sign_z_rt[, 3]


    df_correct$v2_sim_dis_diff_test_sign_z_new  <- v2_sim_dis_diff_test_sign_z_rt[, 1]
    df_correct$v2_sim_dis_diff_test_sign_sq_z_new   <- v2_sim_dis_diff_test_sign_z_rt[, 2]
    df_correct$v2_sim_dis_diff_test_sign_cube_z_new <- v2_sim_dis_diff_test_sign_z_rt[, 3]

    # Now create the models using lmer instead of glmer
    model_abs <- lmer(log_rt_ltm ~ (it_sim_dis_test_z_new + it_sim_dis_test_sq_z_new + it_sim_dis_test_cube_z_new) 
                      + (v2_sim_dis_test_z_new + v2_sim_dis_test_sq_z_new + v2_sim_dis_test_cube_z_new) 
                      + reliability_binary_z_new *  validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))
    
    model_rel <- lmer(log_rt_ltm ~ (it_sim_dis_diff_test_z_new + it_sim_dis_diff_test_sq_z_new + it_sim_dis_diff_test_cube_z_new) 
                      + (v2_sim_dis_diff_test_z_new + v2_sim_dis_diff_test_sq_z_new + v2_sim_dis_diff_test_cube_z_new)
                      + reliability_binary_z_new *  validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))

    model_rel_split <- lmer(log_rt_ltm ~ (it_sim_dis_diff_test_sign_z_new + it_sim_dis_diff_test_sign_sq_z_new+ it_sim_dis_diff_test_sign_cube_z_new) * it_pos_neg 
                            + (v2_sim_dis_diff_test_sign_z_new + v2_sim_dis_diff_test_sign_sq_z_new + v2_sim_dis_diff_test_sign_cube_z_new) * v2_pos_neg 
                            + reliability_binary_z_new * validity_binary_z_new + (1 | participant), 
                      data = df_correct, control = lmerControl(optimizer = "bobyqa"))

    
   


    
    return(list( 
      model_abs       = model_abs,
      model_rel       = model_rel,
      model_rel_split = model_rel_split
    ))
}

In [29]:
results_rt_old_stims <- run_rt_models(df6)
results_rt_new_stims <- run_rt_models(df5)
results_rt_total <- run_rt_models(df_total)

In [30]:
model_rt_abs_old <- summary(results_rt_old_stims$model_abs)
model_rt_abs_new <- summary(results_rt_new_stims$model_abs)
model_rt_abs_total <- summary(results_rt_total$model_abs)

comparison_table_rt_abs <- compare_models(
  model_list = list(model_rt_abs_old, model_rt_abs_new, model_rt_abs_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

# Save the RT comparison tables
writeLines(comparison_table_rt_abs, "comparison_table_distractor_rt_abs.html")

# Relative similarity models
model_rt_rel_old <- summary(results_rt_old_stims$model_rel)
model_rt_rel_new <- summary(results_rt_new_stims$model_rel)
model_rt_rel_total <- summary(results_rt_total$model_rel)

comparison_table_rt_rel <- compare_models(
  model_list = list(model_rt_rel_old, model_rt_rel_new, model_rt_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

writeLines(comparison_table_rt_rel, "comparison_table_distractor_rt_rel.html")

# Split relative similarity models
model_rt_rel_split_old <- summary(results_rt_old_stims$model_rel_split)
model_rt_rel_split_new <- summary(results_rt_new_stims$model_rel_split)
model_rt_rel_split_total <- summary(results_rt_total$model_rel_split)

comparison_table_rt_rel_split <- compare_models(
  model_list = list(model_rt_rel_split_old, model_rt_rel_split_new, model_rt_rel_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)

writeLines(comparison_table_rt_rel_split, "comparison_table_distractor_rt_rel_split.html")

# Display the RT comparison tables

In [31]:
IRdisplay::display_html(comparison_table_rt_abs)
IRdisplay::display_html(comparison_table_rt_rel)
IRdisplay::display_html(comparison_table_rt_rel_split)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_test_z_new 
 (p=0.2857) 
 (p=0.1026) 
 . (p=0.0671) 
 
 
 it_sim_dis_test_sq_z_new 
 (p=0.3887) 
 * (p=0.0339) 
 * (p=0.0267) 
 
 
 it_sim_dis_test_cube_z_new 
 (p=0.2005) 
 ** (p=0.0073) 
 ** (p=0.0039) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 . (p=0.0564) 
 (p=0.1466) 
 * (p=0.0236)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sq_z_new 
 (p=0.1511) 
 * (p=0.0194) 
 * (p=0.0207) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 * (p=0.0452) 
 (p=0.1844) 
 * (p=0.0305)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_sim_dis_diff_test_sign_z_new 
 . (p=0.0884) 
 * (p=0.0321) 
 * (p=0.0186) 
 
 
 v2_sim_dis_diff_test_sign_z_new 
 . (p=0.0762) 
 (p=0.2126) 
 (p=0.9226) 
 
 
 v2_pos_neg 
 . (p=0.0576) 
 (p=0.1013) 
 (p=0.8277) 
 
 
 it_sim_dis_diff_test_sign_sq_z_new:it_pos_neg 
 . (p=0.0614) 
 (p=0.1857) 
 (p=0.8337) 
 
 
 reliability_binary_z_new:validity_binary_z_new 
 * (p=0.0409) 
 (p=0.1346) 
 * (p=0.0280)